In [1]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket
from  data_utils.stochastic_utils.vdistributions.abstract import AbstractDistribution
from  data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, SkewNormalDistribution
from data_utils.stochastic_utils.vdistributions.tools.divergence import kl_divergence_continuous, crps

from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.mfk.skewnd2 import dof_snd_fitter

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [2]:
s = Station("s", 50)
br = PointwiseRecycle(0.8, 1.5)

In [3]:
real_market = DistributiveMarket(
    DistributiveSeries(NormalDistribution(45, 5), NormalDistribution(46, 4), NormalDistribution(30, 5), NormalDistribution(32, 5)),
    DistributiveSeries(NormalDistribution(10, 2), NormalDistribution(11, 3), NormalDistribution(5, 1), NormalDistribution(12, 5)),
    DistributiveSeries(NormalDistribution(10, 3), NormalDistribution(18, 9), NormalDistribution(5, 2), NormalDistribution(13, 6)),
)
per_market = DistributiveMarket(
    DistributiveSeries(NormalDistribution(45, 5), NormalDistribution(46, 4), NormalDistribution(30, 5), NormalDistribution(32, 5)),
    DistributiveSeries(NormalDistribution(10, 2), NormalDistribution(11, 3), NormalDistribution(5, 1), NormalDistribution(12, 5)),
    DistributiveSeries(NormalDistribution(10, 3), NormalDistribution(18, 9), NormalDistribution(5, 2), NormalDistribution(13, 6)),
)

In [4]:
train_aq, train_dp, train_rp = per_market.rvf(1000)
optimied = per_market.submitted_quantity_optimizer(s, br, train_aq, train_dp, train_rp)

In [5]:
print(optimied.x)


[33.01808153 31.63559789 23.4421986  25.549965  ]


In [6]:
real = real_market.rvf(1)
s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4))
# br(real[0].reshape(4), optimied.x, s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4)))

array([464.12449524, 549.18473934, 165.54445507, 475.69107181])

In [7]:
numpy.sum(br(
    real[0], 
    optimied.x, 
    s.trade(real[0].reshape(4), optimied.x, real[1].reshape(4), real[2].reshape(4))
))

1654.544761463012

In [8]:
numpy.sum(br(
    real[0], 
    real[0].reshape(4), 
    s.trade(real[0].reshape(4), real[0].reshape(4), real[1].reshape(4), real[2].reshape(4))
))

1530.035883552644

In [14]:
numpy.concatenate((per_market.crps(*real), [per_market.price_kl_divergence()])).reshape(-1)

array([3.86949392, 1.28545611, 2.37841335, 2.82176832, 1.37862214,
       0.80720281, 0.75125449, 3.90868115, 1.34504643, 2.76019019,
       1.85661905, 4.40265453, 0.12768733, 0.95663698, 0.31814718,
       0.04343267])

In [10]:
print(per_market.price_kl_divergence())

[0.12768733 0.95663698 0.31814718 0.04343267]
